### Import

In [ ]:
from langchain_core.language_models.fake_chat_models import (
    FakeListChatModel, 
    FakeMessagesListChatModel
)
from langchain_core.runnables import ConfigurableField
from langchain_core.output_parsers import (
    JsonOutputParser, 
    StrOutputParser, 
    CommaSeparatedListOutputParser,
    NumberedListOutputParser,
    PydanticOutputParser
)
import json
from pydantic import (
    BaseModel,
    Field
)
from langchain_core.messages import (
    AIMessage,
    ToolCall,
    ToolMessage
)
from langchain_core.messages.tool import tool_call
from langchain_core.tools import (
    tool,
    StructuredTool,
    BaseTool
)

### Test Model

In [ ]:
resp_list = [
    'Hello from AI', 
    'How may I help you?', 
    'Another message from AI']
model = FakeListChatModel(responses = resp_list)

In [ ]:
resp = model.invoke('')
print(resp.__class__)
print(resp.content)
for resp in resp_list[1:]:
    print(model.invoke('').content)

In [ ]:
model2 = model.configurable_fields(
    responses=ConfigurableField(
        id="responses",
        name="List of responses to cycle through in order.",
        description="List of responses to cycle through in order.",
    )
)

In [ ]:
custom_resp = ['Custom AI message']
config = {"configurable": {"responses": custom_resp}}
resp = model2.invoke("", config=config)
print(resp.__class__)
print(resp.content)

### With Parser

In [ ]:
chain = model | StrOutputParser()
resp = chain.invoke('')
print(resp.__class__)
print(resp)

for resp in resp_list[1:]:
    print(chain.invoke(''))

In [ ]:
custom_resp = '{"price": "$1,000", "RAM": "6GB"}'
# custom_resp = json.dumps({'price': '$1,000', 'RAM': '6GB'})
config = {"configurable": {"responses": [custom_resp]}}
chain2 = model2 | JsonOutputParser()
resp = chain2.invoke("", config=config)
print(resp.__class__)
print(resp)

In [ ]:
custom_resp = 'one, two, three'
config = {"configurable": {"responses": [custom_resp]}}
chain2 = model2 | CommaSeparatedListOutputParser()
resp = chain2.invoke("", config=config)
print(resp.__class__)
print(resp)

In [ ]:
custom_resp = """
1. One
2. Two
3. Three
"""
config = {"configurable": {"responses": [custom_resp]}}
chain2 = model2 | NumberedListOutputParser()
resp = chain2.invoke("", config=config)
print(resp.__class__)
print(resp)

### Pydantic Parser

In [ ]:
class AnswerWithJustification(BaseModel):
    '''An answer to the user question along with justification for the answer.'''
    answer: str
    justification: str

In [ ]:
data = {
    'answer': 'They weigh the same',
    'justification': (
        'Both a pound of bricks and a pound of feathers weigh one pound. '
        'The weight is the same, but the volume and density of the two substances differ.'
    )
}
config = {"configurable": {"responses": [json.dumps(data)]}}
chain2 = model2 | PydanticOutputParser(pydantic_object=AnswerWithJustification)
resp = chain2.invoke("", config=config)
print(resp.__class__)
print(json.dumps(resp.__dict__, indent=4))

### Tools

In [ ]:
@tool
def add_number_tool(x: int, y: int) -> int:
    """Add two numbers"""
    return x + y

print(add_number_tool.__class__)

res = add_number_tool.invoke({"x": 2, "y": 3})
print(res.__class__)

print(res)

In [ ]:
class IncreasePricesInput(BaseModel):
    prices: list[float] = Field(description="List of prices to increase")
    increase_factor: float = Field(description="Factor by which to increase the prices")

@tool(args_schema=IncreasePricesInput)
def increase_prices(prices: list[float], increase_factor: float) -> list[float]:
    """Increase a list of prices by multiplying them with an increase factor"""
    return [round(price * increase_factor, 2) for price in prices]

print(increase_prices.__class__)

res = increase_prices.invoke({ "prices": [2.5,2.8,3.3], "increase_factor": 1.5})
print(res.__class__)
print(res)

In [ ]:
class MultiplyInput(BaseModel):
    x: int = Field(description="Number")
    y: int = Field(description="Another number")

def multiply_number(x: int, y: int) -> int:
    """Multiply 2 numbers"""
    return x * y

multiply_number_tool = StructuredTool.from_function(
    func=multiply_number,
    name="Multiplication",
    description="Multiply 2 numbers",
    args_schema=MultiplyInput,
    return_direct=True,
)

res = multiply_number_tool.invoke({"x": 2, "y": 3})
print(res.__class__)
print(res)

### Tool Message

In [ ]:
tools: list[BaseTool] = {
    "multiply_number_tool" : multiply_number_tool,
    "increase_prices" : increase_prices,
    "add_number_tool" : add_number_tool
}

In [ ]:
# An example of model responded with tool calls
ai_msg = AIMessage(
    content = "",
    tool_calls = [
            tool_call(
                name="multiply_number_tool", 
                args = {"x": 2, "y": 3}, 
                id = "tool_call_id_1"),
        ]
)

In [ ]:
# return responses for tool calls
def run_tool(msg: AIMessage, tools: list[BaseTool]) -> list[ToolMessage]:
    tool_messages: list[ToolMessage] = []
    for tool in msg.tool_calls:
        tool_name = tool["name"]
        if tool_name in tools:
            res = tools[tool_name].invoke(tool["args"])
            tool_messages.append(ToolMessage(
                content=res,
                artifact={},
                tool_call_id=tool["id"],
            ))

    return tool_messages

In [ ]:
messages = run_tool(ai_msg, tools)
for msg in messages:
    print(msg.content)